# Testing a Black Box: Detecting Bias in a Medical Risk Score Algorithm
---

ML Failures lab: Dissecting Racial Bias in a Medical Risk Score by Nick Merrill, Inderpal Kaur, Samuel Greenberg is licensed under [CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0).

This lab is based off of [Dissecting racial bias in an algorithm used to manage the health of populations](https://science.sciencemag.org/content/366/6464/447) by Ziad Obermeyer et al (2019).


# Background

To effectively manage patients, health systems often need to estimate patients' health risks. By calculating numerical "risk scores" for patients, healthcare providers can prioritize patients who are most at-risk and allocate more resources to them.

In this lab, we examine an algorithm widely-used in industry to assign medical risk scores to patients. For each patient, the algorithm uses *medical cost* (i.e., the amount a patient spends on medical care) to calculate their **risk score**. Some other features the model also takes in include details about the patient's medical history.

We've received some reports of concerns about racial bias in AI risk scoring systems, so want to test the AI to check for bias. However, we can't see the algorithm or data because this is a proprietary model trained on private medical information. What we can do is send inputs (hypothetical patient data) to the model and see the corresponding outputs (the hypothetical patient's risk score).

Let's see what we can uncover about this black box model just by **testing various inputs**!

In [ ]:
#@title Preparation: click the play button to load the input data we use to test the AI.

%%capture
# import statements
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker
from statsmodels.nonparametric.smoothers_lowess import lowess
%matplotlib inline

#utility function
def convert_to_percentile(df, col_name):
    """
    Convert df column col_name to percentile.
    """
    return pd.qcut(df[col_name].rank(method='first'), 100, labels=range(1, 101))

# first, let's load the data we will need
# download this data at 
# https://github.com/daylight-lab/mlfailures/blob/master/health-care-bias-lab.csv
!wget https://raw.githubusercontent.com/daylight-lab/mlfailures/master/health-care-bias-lab.csv
data = pd.read_csv('health-care-bias-lab.csv')

#add a column of risk percentiles to the dataframe called 'risk_percentile'
risk_percentile = convert_to_percentile(data, "risk_score_t")
data["risk_percentile"] = risk_percentile
data["race"] = data["race"].str.replace("black", "Black").replace("white", "White")

data_subset = data[["risk_percentile", "race", "cost_t", "gagne_sum_t"]].rename({"risk_percentile": "Risk Score", 
                                                                                 "race": "Race", "cost_t": "Medical Cost", 
                                                                                 "gagne_sum_t": "Illness"}, axis=1)
inputs = data_subset[["Race", "Medical Cost", "Illness"]]
outputs = data_subset[["Risk Score"]]

In [ ]:
#@title Data Preview: click the play button to view the data in detail.

print("The inputs we sent to the model...", "\n")
print(inputs.head(), "\n")
print("The risk score outputs we got back from the model...", "\n")
print(outputs.head())

## Medical Cost and Risk Score

Since the AI is using medical cost to calculate the risk scores, we expect patients with higher medical costs to have higher risk scores. If our algorithm is fair, the relationship between a patient's input cost and their output risk score shouldn't change based on the patient's race. In other words, patients with the same cost should have similar risk scores regardless of their race.

To check this relationship, we can show the input cost and the output risk on a chart like the one below.

In [ ]:
#@title Click play to display a chart of the relationship between cost and risk score.

# fit a LOWESS (Locally Weighted Scatterplot Smoothing) model to the scatterplot above for each race
def LowessForScoreToCost(data):
  return lowess(np.array(data['Medical Cost']), np.array(data['Risk Score']), it=35, frac=0.2, delta=2)[:, 1]


#create dataframe with the average total medical expenditure for each race at each risk percentile
group_cost = data_subset.groupby(["Risk Score", "Race"])[["Medical Cost"]].mean().reset_index()

#divide group_cost into two dataframes based on race
group_cost_black = group_cost[group_cost['Race'] == 'Black']
group_cost_white = group_cost[group_cost['Race'] == 'White']

# Calculate the curve for group "black" by LOWESS 
lowess_black = LowessForScoreToCost(group_cost_black)

## TODO：Please change this line of code to make the result correct!
## "correct" means you can see another curve in orange showing in the chart generated below.
lowess_white = np.zeros(len(group_cost_white))


# Plot the scatter chart
plt.figure(figsize=(12,7))
ax = sns.scatterplot(x = "Risk Score", y = "Medical Cost", data = group_cost, hue = "Race", marker = "x")
plt.yscale('log')
plt.plot(group_cost_black['Risk Score'], lowess_black)
plt.plot(group_cost_white['Risk Score'], lowess_white)
ax.set_yticks([1000, 3000, 8000, 20000, 60000])
ax.get_yaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())

**QUESTIONS:**
1. If you were to pick a Black patient and a white patient from this dataset, both with identical medical costs, would you expect the algorithm to systematically assign a higher risk score to one or the other?
2. Can you conclude from this data alone that the model is fair or not fair?

## Chronic Illness and Risk Score

The purpose of the risk score is to identify patients who will likely need high-risk care or have more illnesses that require medical attention. This means that patients with more illnesses should have higher risk scores.

Again, we want to make sure that patients with the same level of illness get the same risk scores regardless of race. Let's make another plot like the one above to show these relationships.



In [ ]:
#@title Click play to display a chart of the relationship between number of chronic illnesses and risk score...

# Fit a Generalized Linear Model (GLM) to the scatterplot above for each race
def Glm(data):
  X_b = sm.add_constant(np.array(data["Risk Score"]))
  glm = sm.GLM(data["Illness"], X_b, family=sm.families.Gaussian(link=sm.families.links.log())).fit()
  return glm.predict()


# Create dataframe with the average number of chronic illnesses for each race at each risk percentile
group_illness = data_subset.groupby(["Risk Score", "Race"])[["Illness"]].mean().reset_index()

# Divide the grouped dataframe into two dataframes based on race
group_illness_black = group_illness[group_illness['Race'] == 'Black']
group_illness_white = group_illness[group_illness['Race'] == 'White']

# Calculate the curve for group "white" by GLM
illness_white = Glm(group_illness_white)

## TODO：Please change this line of code to make the result correct!
## "correct" means you can see the blue line becomes a curve along with those blue data points in the chart.
illness_black = np.zeros(len(group_illness_black))

# Plot the model on the scatter chart
plt.figure(figsize=(12,7))
sns.scatterplot(x="Risk Score", y="Illness", data=group_illness, hue="Race", marker="x")
plt.plot(group_illness_black["Risk Score"], illness_black)
plt.plot(group_illness_white["Risk Score"], illness_white)

**QUESTIONS:**
1. If you were to pick a Black patient and a white patient from this dataset, both with identical number of chronic illnesses, would you expect the algorithm to systematically assign a higher risk score to one or the other?
2. Can you conclude from this data alone that the model is fair or not fair?

## What Went Wrong?

The charts above show us that a Black patient and a White patient who have the same medical cost will get similar risk scores from the AI system. However, a Black patient and a White patient who have the same number of illnesses can get very different risk scores. Specifically, the Black patient will be predicted to have a lower risk score than the white patient. 

Imagine: a Black patient and a white patient walk into the same hospital at the same time. Their medical records are identical. The algorithm would recommend that the white patient be triaged before the Black patient! 

How did this happen? 

To get to the bottom of this effect, let's look at the relationship between medical cost and number of illness, by race.

In [ ]:
#@title Click play to display a chart of the relationship between number of chronic illnesses and medical cost...

#fit a LOWESS (Locally Weighted Scatterplot Smoothing) model to the scatterplot above for each race
def LowessForCostToIllness(data):
  return lowess(np.array(data['Medical Cost']), np.array(data['Illness (%-tile)']), it=35, frac=0.3, delta=2)[:, 1]

#add a column of illness percentiles to the dataframe called 'illness_percentile'
illness_percentile = convert_to_percentile(data_subset, "Illness")
data_subset['Illness (%-tile)'] = illness_percentile
#create dataframe with the average total medical expenditure for each race at each illness percentile
illnesses = data_subset.groupby(['Illness (%-tile)', "Race"])[["Medical Cost"]].mean().reset_index()

#divide illnesses into two dataframes based on race
illness_b = illnesses[illnesses['Race'] == 'Black']
illness_w = illnesses[illnesses['Race'] == 'White']


# Calculate the curve for group "white" by LOWESS
white_lowessC2I = LowessForCostToIllness(illness_w)

## TODO：Please change this line of code to make the result correct!
## "correct" means you can see another curve in orange showing in the chart generated below.
black_lowessC2I = np.zeros(len(illness_b))


# Plot the model on the scatter chart
plt.figure(figsize=(12,7))
ax = sns.scatterplot(x = "Illness (%-tile)", y = "Medical Cost", data = illnesses, hue = "Race", marker = "x")
plt.plot(illness_b["Illness (%-tile)"], black_lowessC2I)
plt.plot(illness_w["Illness (%-tile)"], white_lowessC2I)
plt.yscale('log')
ax.set_yticks([1000, 3000, 8000, 20000, 60000])
ax.get_yaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())

**Questions:**
1. If you were to pick a Black patient and a white patient from this dataset, both with an identical number of chronic illnesses, would you expect either patient to systematically spend more on their health care costs?
2. How did this discrepancy caused the algorithm to underestimate the medical risk to Black patients?
3. Why might we observe this discrepancy?



## Conclusions and takeaways

The AI system used to calculate patient risk scores might be a black box, but we were still able to find evidence of bias in the model by testing it with different inputs and displaying the relationship with the output.

The takeaway here is clear: we can identify bias in black-box models---*as long as we have access to the sensitive features.*

When we drilled down into this bias, we discovered a cause: given a Black patient and a white patient with the same number of chronic illnesses, the white patient will spend more on their health care. But why? Obermeyer et al. opined in the Nature paper on which this lab is based:
 
>The literature broadly suggests two main potential channels. First, poor patients face substantial barriers to accessing health care, even when enrolled in insurance plans. Although the population we study is entirely insured, there are many other mechanisms by which poverty can lead to disparities in use of health care: geography and differential access to transportation, competing demands from jobs or child care, or knowledge of reasons to seek care (1-3). To the extent that race and socioeconomic status are correlated, these factors will differentially affect Black patients. Second, race could affect costs directly via several channels: direct (“taste-based”) discrimination, changes to the doctor–patient relationship, or others. A recent trial randomly assigned Black patients to a Black or White primary care provider and found significantly higher uptake of recommended preventive care when the provider was Black (4). This is perhaps the most rigorous demonstration of this effect, and it fits with a larger literature on potential mechanisms by which race can affect health care directly. For example, it has long been documented that Black patients have reduced trust in the health care system (5), a fact that some studies trace to the revelations of the Tuskegee study and other adverse experiences (6). A substantial literature in psychology has documented physicians’ differential perceptions of Black patients, in terms of intelligence, affiliation (7), or pain tolerance (8). **Thus, whether it is communication, trust, or bias, something about the interactions of Black patients with the health care system itself leads to reduced use of health care. The collective effect of these many channels is to lower health spending substantially for Black patients, conditional on need—a finding that has been appreciated for at least two decades (9).**

## Reflection Questions

Here is a final open-ended question for you to answer.

1. How could we use the data we have to create new proxies for health needs that may be less biased than medical costs?

## References

1. K. Fiscella, P. Franks, M. R. Gold, C. M. Clancy, JAMA 283, 2579–2584 (2000).
2. N. E. Adler, K. Newman, Health Aff. 21, 60–76 (2002).
3. N. E. Adler, W. T. Boyce, M. A. Chesney, S. Folkman, S. L. Syme, JAMA 269, 3140–3145 (1993).
4. M. Alsan, O. Garrick, G. C. Graziani, “Does diversity matter for health? Experimental evidence from Oakland” (National Bureau of Economic Research, 2018).
5. K. Armstrong, K. L. Ravenell, S. McMurphy, M. Putt, Am. J. Public Health 97, 1283–1289 (2007).
6. M. Alsan, M. Wanamaker, Q. J. Econ. 133, 407–455 (2018).
7. M. van Ryn, J. Burke, Soc. Sci. Med. 50, 813–828 (2000).
8. K. M. Hoffman, S. Trawalter, J. R. Axt, M. N. Oliver, Proc. Natl. Acad. Sci. U.S.A. 113, 4296–4301 (2016).
9. J. J. Escarce, F. W. Puffer, in Racial and Ethnic Differences in the Health of Older Americans (National Academies Press, 1997), chap. 6; www.ncbi.nlm.nih.gov/books/ NBK109841/.

<!-- # Feedback

**Instructors**: Please [provide feedback](https://docs.google.com/forms/d/1UuUVBBMTU_2aMvzsGnTR_4i1w3F6tLaaqdIr7dQrgSI/edit?ts=5efa771b&dods) to help improve this lab.

**Students**: Please [provide feedback](https://docs.google.com/forms/d/1jI8oXRkqD1l1ARuZR1y9W_qkOystPr-YEyywNDez46M/edit?ts=5efa772a&dods) to help improve this lab. -->
